In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0,org.apache.spark:spark-sql-kafka-0-10_2.10:2.2.3,org.elasticsearch:elasticsearch-hadoop:7.6.2 pyspark-shell'

In [2]:
#org.apache.spark:spark-sql-kafka-0-10_2.10:2.2.3
#instead of org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0

In [3]:
from pyspark import SparkContext,SparkConf
#    Spark Streaming
from pyspark.streaming import StreamingContext
from pyspark.sql.session import SparkSession
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json
import nltk
import logging
from datetime import datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [4]:
def evaluate_sentiment(avg):
    try:
        if avg < 0:
            return 'Negative'
        elif avg > 0:
            return 'Positive'
        else:
            return 'Neutral'
    except TypeError:
        return 'Neutral'
    
eval_udf = udf(evaluate_sentiment,StringType())

In [5]:
def start_stream(df):
    df.writeStream.outputMode('append').format('org.elasticsearch.spark.sql')\
    .option('checkpointLocation','/checkpoint')\
    .option("es.nodes.wan.only",True)\
    .option('es.nodes','localhost').option('es.port',9200)\
    .option('es.spark.sql.streaming.sink.log.enabled',False).start('twitter_stream_analysis/doc').awaitTermination()

In [6]:
conf = SparkConf().setAppName('twitter_analysis')
spark = SparkSession.builder.config("spark.es.nodes.wan.only","true").appName('twitter_analysis').getOrCreate()
conf.set("es.index.auto.create", "true")
#conf.set("es.net.ssl","true")
#conf.set("es.nodes.wan.only","true")

In [7]:
schema = StructType([StructField("date", StringType(), True),
                    StructField("user", StringType(), True),
                    StructField("text", StringType(), True),
                    StructField("reply_count", IntegerType(), True),
                    StructField("retweet_count", IntegerType(), True),
                    StructField("favorite_count", IntegerType(), True),
                    StructField("sentiment_score", DoubleType(), True)])

In [8]:
logging.getLogger("py4j").setLevel(logging.ERROR)

In [9]:
kafkaStream = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "kafkaspark").option('failOnDataLoss',False).load()

In [10]:
esTimeStampFormat = "yyyy-MM-dd'T'HH:mm:ss.SSSZ"

In [11]:
parsed_df = kafkaStream.select(from_json(col('value').cast('string'),schema).alias('parsed_value'))\
            .withColumn('timestamp', date_format(to_timestamp(lit(current_timestamp()),"yyyy-MM-dd'T'HH:mm:ss.SSSZ"),esTimeStampFormat))

df = parsed_df.select('parsed_value.*', 'timestamp')

In [12]:
evaluated_df = df.withColumn('status',eval_udf('sentiment_score'))\
               .withColumn('date',date_format(to_timestamp(col('timestamp'),"yyyy-MM-dd'T'HH:mm:ss.SSSZ"),esTimeStampFormat))

In [13]:
start_stream(evaluated_df)

Py4JJavaError: An error occurred while calling o88.start.
: java.io.IOException: Mkdirs failed to create file:/checkpoint (exists=false, cwd=file:/Users/ashwin/Documents/Fall2020/IDS561/TwitterKafkaES)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:455)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:440)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:911)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:892)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:789)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:778)
	at org.apache.spark.sql.execution.streaming.StreamMetadata$.write(StreamMetadata.scala:76)
	at org.apache.spark.sql.execution.streaming.StreamExecution$$anonfun$2.apply(StreamExecution.scala:124)
	at org.apache.spark.sql.execution.streaming.StreamExecution$$anonfun$2.apply(StreamExecution.scala:122)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.execution.